In [24]:
from tqdm import tqdm
from bitstring import BitArray
import numpy as np

In [55]:
def test_hash_int(value):
    
    phi = (1 + 5 ** 0.5) / 2
    g = int(2 ** 64 /phi)
    value ^= value >> 61 # Does a "bitwise exclusive or". Each bit of the output is the same as the corresponding bit in x if that bit in y is 0, and it's the complement of the bit in x if that bit in y is 1.
    value = (g * value) >> 61
    return value

In [35]:
def test_hash_float(value):
    
    value = BitArray(float=value, length=64)
    phi = (1 + 5 ** 0.5) / 2
    g = int(2 ** 64 /phi)
    

    value ^= value >> 61
    value = int(g * value.float)
 
    return value

In [45]:
setm = set()
for i in tqdm(np.arange(0,2000, 0.001)):
    setm.add(test_hash_float(i))


100%|██████████| 2000000/2000000 [01:04<00:00, 30931.11it/s]


In [47]:
print(len(setm))
print(len(np.arange(0,2000,0.001)))

2000000
2000000


In [53]:
test_hash_float(0.001144545545555554423)

13048637362605560

In [56]:
def minhash(floatValues, intValues):
    floatVal_h = []
    intVal_h = []
    
    for fl in floatValues:
        floatVal_h.append(test_hash_float(fl))
        
    for it in intValues:
        intVal_h.append(test_hash_int(it))
        
    #.... e mo? da valutare....
    